### Описание проекта
Проект для «Викишоп» с BERT.

Описание исследования: Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию

Цель исследования: Обучить модель классифицировать комментарии на позитивные и негативные на основе набора данных с разметкой о токсичности правок.

Критерии, которые важны заказчику:
Построить модель со значением метрики качества F1 не меньше 0.75.

Задачи исследования:

Загрузить данные.
Проанализировать данные.
Обучить разные модели с различными гиперпараметрами. 
Проверить данные на тестовой выборке и сделать выводы.

Данное исследование разделим на несколько частей.

Часть 1. Изучениеm общей информации:

Изучение файлов с данными, получение общей информации, загрузка библиотек. 

Часть 2. Подготовка данных:

Нахождение и ликвидация пропусков. Проверка и ликвидация дубликатов. Исследовательский анализ данных.

Часть 3. Обучение моделей:

Обучение моделей LightGBM, LogisticRegression, DesicionTreeClassifier, KNN. Нахождение лучшей модели по метрике F1 на кросс-валидации.

Часть 4. Тестирование лучшей модели на тестовой выборке и итоговые выводы.

#### 1. Изучение общей информации

In [1]:
#!pip install torch

In [2]:
#!pip install transformers

In [3]:
#!pip install lightgbm

In [4]:

import pandas as pd
import numpy as np
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from transformers import BertTokenizer
from transformers import BertConfig
from transformers import BertModel
from sklearn.model_selection import train_test_split

In [5]:
RANDOM_STATE = 42

In [6]:
data=pd.read_csv('C:\\Users\\N\\Documents\\Документы\\Data science\\МО для текстов\\toxic_comments.csv')
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


#### 2. Подготовка данных

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


 Пропусков в данных нет.

In [8]:
data.duplicated().sum()

0

In [9]:
data['text'].duplicated().sum()

0

Полных и неполных дубликатов в данных нет. Проверим целевой признак на сбалансированность классов.

In [10]:
data['toxic'].value_counts(normalize=True) 

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Данные очевидно несбалансированы. Это будет необходимо учесть при делении выбоки на обучающую и тестовую.

Построим векторы текстов с помощью предобученной модели BERT.
Для применения модели Bert необходимо уменьшить размер выборки, иначе она будет очень долго обрабатываться.
Для корректной работы модели необходимо, чтобы количество токенов было не более 512, для этого оставим в выборке только строки с количеством символов не более 1800 (тк одно слово обычно содержит около 3-4 токенов) и выберем из них 2000 произвольных строк (пробовала 10000 строк - считалось 1,5 часа и метрика F1 получалась лучше на 0.01-0.02 на кросс-валидации и тестовой выборке соответственно).

In [11]:
data['len'] = data['text'].str.len()
data_bert = data.query('len < 1800')

In [12]:

data_bert = data_bert.sample(2000).reset_index(drop=True)

Применим предобученную модель 'unitary/toxic-bert'.
Инициализируем токенизатор как объект класса BertTokenizer.
Преобразуем текст в номера токенов из словаря методом encode().
Применим метод padded, чтобы после токенизации длины исходных текстов в корпусе были равными. 
Отбросим нулевые токены и «создадим маску» для действительно важных токенов, то есть укажем нулевые и не нулевые значения.

In [13]:
tokenizer = BertTokenizer.from_pretrained('unitary/toxic-bert')

tokenized = data_bert['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)
print(padded.shape)
print(attention_mask.shape)

(2000, 447)
(2000, 447)


Инициализируем конфигурацию BertConfig. Передадим ей файл с предобученной моделью и конфигурацией.

In [14]:
config = BertConfig.from_pretrained('unitary/toxic-bert')
model = BertModel.from_pretrained('unitary/toxic-bert', config=config)

Преобразуем текст в эмбеддинги.
Чтобы хватило оперативной памяти, сделаем размер батча 100.
Преобразуем данные в формат тензоров - многомерных векторов в библиотеке torch.
Передадим модели данные и маску. 
Для ускорения вычисления функцией no_grad() в библиотеке torch укажем, что градиенты не нужны.

In [15]:
batch_size = 100
embeddings = []


for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/20 [00:00<?, ?it/s]

Соберём все эмбеддинги в матрицу признаков.

In [16]:
features = np.concatenate(embeddings)


Разделим выборку на обучающую и тестовую с учетом стратификации.

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    features,
    data_bert['toxic'],
    test_size = 0.25, 
    random_state = 42, stratify=data_bert['toxic'])
model=LogisticRegression(random_state=RANDOM_STATE)

#### 3. Обучение моделей

Обучим 4 разных модели: KNN, DecisionTreeClassifier, LogisticRegression, LightGBM, с помощью пайплайна зададим разные гиперпараметры и применим RandomSearchCV для поиска лучшей метрики F1_score на кросс-валидации.

In [18]:
pipe_final = Pipeline([
    #('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

In [19]:
param_distributions = [
    
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2,7),
           
    },
   
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 10),
        'models__min_samples_leaf': range(1, 6),
        'models__min_samples_split': range(2, 6),
           
    },
        
   
    {
        'models': [LogisticRegression(random_state=RANDOM_STATE, solver='liblinear')],
        'models__C': range(1,5),
        'model__penalty': ['l1', 'l2']
       },
    {

         'models': [lgb.LGBMRegressor(learning_rate=0.1, random_state=RANDOM_STATE)],
         'models__n_estimators': [100,300],
         'models__max_depth': range(1,3),
    }

        
] 

In [20]:
randomized_search = RandomizedSearchCV(
    pipe_final, 
    param_distributions, 
    cv=5,
    scoring='f1', 
    n_jobs=-1,
    random_state=RANDOM_STATE
)

In [21]:
randomized_search.fit(X_train, y_train)
print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print ('Метрика лучшей модели на кросс-валидации:', randomized_search.best_params_)
print ('Метрика лучшей модели на кросс-валидации:', round(randomized_search.best_score_,2))

C:\ProgramData\anaconda3\envs\practicum\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.91615602 0.91098492 0.91977184 0.91987751 0.91098492 0.92682343
 0.91977184        nan 0.91146122 0.91977184]
  warnings.warn(


Лучшая модель и её параметры:

 Pipeline(steps=[('models',
                 DecisionTreeClassifier(max_depth=2, min_samples_leaf=3,
                                        min_samples_split=5,
                                        random_state=42))])
Метрика лучшей модели на кросс-валидации: {'models__min_samples_split': 5, 'models__min_samples_leaf': 3, 'models__max_depth': 2, 'models': DecisionTreeClassifier(max_depth=2, min_samples_leaf=3, min_samples_split=5,
                       random_state=42)}
Метрика лучшей модели на кросс-валидации: 0.93


Итак, лучшая модель - DecisionTreeClassifier(max_depth=3, min_samples_leaf=2, min_samples_split=3,random_state=42) с метрикой F1=0.93.
Протестируем данную модель на тестовой выборке.

#### 4. Тестирование лучшей модели на тестовой выборке и итоговые выводы.

In [22]:
model_best = randomized_search.best_estimator_
predictions = model_best.predict(X_test)
f1_test = f1_score(y_test, predictions)
print(f"F1 на тестовой выборке: {f1_test:.2f}")

F1 на тестовой выборке: 0.92


Итак, в соответствии с задачами исследования были проделаны следующие этапы работы:

Загружены данные, загружены необходимые библиотеки.

Данные были проанализированы на предмет пропусков и дубликатов (не выявлено). Но был выявлен дисбаланс классов в целевом признаке - 90:10%.

Были построены векторы текстов с помощью предобученной модели 'unitary/toxic-bert'.
Тестовая выборка была стратифицирована наравне с обучающей.

Были обучены модели LightGBM, LogisticRegression, DesicionTreeClassifier, KNN с различными гиперпараметрами. 
По итогам расчета метрики F1= 0.93 на кросс-валидации была выбрана лучшая модель - DecisionTreeClassifier(max_depth=3, min_samples_leaf=2,
                                        min_samples_split=3, random_state=42))

На тестовой выборке данная модель показала результат F1=0.92, что полностью соответсвует критерию заказчика (построить модель со значением метрики качества F1 не меньше 0.75).